fai una join su id
prima cosa dividi tra camel e kebab (tieni traccia di chi sta rispondendo)
dividi per cs background 
calcola accuracy (numeri di risposte corrette / il numero di risposte totali) per ogni persona di ogni gruppo e poi fai la media intra gruppo : cosi ho l'accuracy media
e poi lo fai per ogni cs background
tenendo conto di solo le risposte corrette, calcoli il tempo medio di risposta di ogni persona di ogni gruppo
poi fai il paired t test per vedere se ci sono differenze significative tra i gruppi

In [16]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('code-comprehension.runs.csv')

# Loop through columns containing 'questionId' and apply the mapping
for i in range(20):  # Iterate over the 20 questionId columns
    column_name = f'results[{i}].questionId'  # Dynamically format the column name
    if column_name in df.columns:  # Ensure the column exists
        # Map the values based on the conditions
        df[column_name] = np.where(
            df[column_name].between(0, 9),  # Condition for camelCase
            'camelCase',
            np.where(
                df[column_name].between(10, 19),  # Condition for kebab-case
                'kebab-case',
                df[column_name]  # Keep the original value if not in range
            )
        )

# Save the updated data back to CSV
df.to_csv('code-comprehension.runs.csv', index=False)

In [18]:
import pandas as pd

def transform_user_data(file_path, output_path):
    # Load the dataset
    data = pd.read_csv(file_path)

    # Identify user-specific and demographic columns
    user_cols = ['_id', 'demographics.age', 'demographics.csBackground',
                 'demographics.yearsOfExperience']
    question_cols = [col for col in data.columns if col.startswith('results')]

    # Separate the data for reshaping
    user_data = data[user_cols]
    question_data = data[question_cols]

    # Melt the question-related data to long format
    melted = question_data.melt(ignore_index=False, var_name='question_metric', value_name='value')

    # Extract question number and metric type (elapsedTime, isCorrect, questionId)
    melted['question'] = melted['question_metric'].str.extract(r'results\[(\d+)\]')
    melted['metric'] = melted['question_metric'].str.extract(r'results\[\d+\]\.(.+)')

    # Pivot to have one row per question with columns for elapsedTime, isCorrect, questionId
    reshaped = melted.pivot_table(index=[melted.index, 'question'], columns='metric', values='value', aggfunc='first').reset_index()

    # Merge back with user demographic data
    reshaped = reshaped.merge(user_data, left_on='level_0', right_index=True).drop(columns=['level_0'])

    # Rename columns for clarity
    reshaped = reshaped.rename(columns={
        'question': 'question_number',
        'elapsedTime': 'elapsed_time',
        'isCorrect': 'is_correct',
        'questionId': 'question_id'
    })

    reshaped['question_id'] = reshaped['question_id'].replace(20, 'camelCase')

    # Convert data types where necessary
    reshaped['question_number'] = reshaped['question_number'].astype(int)
    reshaped['elapsed_time'] = pd.to_numeric(reshaped['elapsed_time'], errors='coerce')
    reshaped['is_correct'] = reshaped['is_correct'].astype('bool', errors='ignore')

    # print(reshaped.head(20))

    # Save the transformed data to a CSV file
    reshaped.to_csv(output_path, index=False)
    
transform_user_data('code-comprehension.runs.csv', 'transformed_data.csv')